In [1]:
from f_train import *

### load data

In [2]:
raw_data = np.loadtxt('../dmel_data.txt').T  # transpose the data so that the shape is 6078 cells x (3+6*99) columns
gene_exp = np.copy(raw_data[:,3:])  # excluding (x,y,z) coordinates)
n_cell = gene_exp.shape[0]  # number of cells
print("# of cells: ", n_cell)

# of cells:  6078


In [3]:

def imputation(tr, te):
    complete_all = ([int(x) - 1 for x in open('../indices_complete.txt','r').readline().split()])
    comp_ind = list(map(int, list((np.array(complete_all)[::6]-3)/6)))
    gene_sel = np.sort(np.random.choice(comp_ind, 3, replace=False))
    print("selected genes: ", gene_sel)
    mt_list=[]
    tp_list=[]
    save_tr=[]
    save_te=[]
    for i in range(len(gene_sel)):
        mt = np.random.randint(1, 6, 1)
        mt_list.append(mt)
        tp = np.random.choice(range(1,7), mt, replace=False)
        tp_list.append(tp)
        print("# of missing time points: ", mt, " which time point is missing: ", tp)
        for j in tp:
            save_tr.append(np.copy(tr[(3039*(j-1)):(3039*j), gene_sel[i]]))
            save_te.append(np.copy(te[(3039*(j-1)):(3039*j), gene_sel[i]]))
            tr[(3039*(j-1)):(3039*j), gene_sel[i]] = 0
            te[(3039*(j-1)):(3039*j), gene_sel[i]] = 0
    comp_ind_ = [i for i in comp_ind if i not in gene_sel]
    print(len(comp_ind), len(comp_ind_), comp_ind_)
    corr, r2 = nn_complete(tr, te, comp_ind_)
    imputed_tr = []
    imputed_te = []
    for i in range(len(gene_sel)):
        for j in tp_list[i]:
            imputed_tr.append(tr[(3039*(j-1)):(3039*j), gene_sel[i]])
            imputed_te.append(te[(3039*(j-1)):(3039*j), gene_sel[i]])
    tr_bin = np.vsplit(tr, 6)
    te_bin = np.vsplit(te, 6)
    data_complete = [np.vstack((tr_bin[i], te_bin[i])) for i in range(6)]
    data_complete = np.vstack(data_complete)
    data_complete[data_complete < 0] = 0
    med = np.median(data_complete, axis=0)
    
    return gene_sel, med

In [ ]:
res = []
for i in range(10):
    train_cell_ind = np.random.choice(range(n_cell), size = int(n_cell/2), replace=False)
    test_cell_ind = [ i for i in range(n_cell) if i not in set(list(train_cell_ind))]
    tr_data = np.vstack([gene_exp[train_cell_ind, i::6] for i in range(6)]) #(3039 cells * 6 timepoints) x 99 genes
    te_data = np.vstack([gene_exp[test_cell_ind, i::6] for i in range(6)])  #(3039 cells * 6 timepoints) x 99 genes
    gene, med = imputation(tr_data, te_data)
    res.append([gene,med])

selected genes:  [29 42 60]
# of missing time points:  [5]  which time point is missing:  [3 5 4 6 2]
# of missing time points:  [4]  which time point is missing:  [6 3 4 1]
# of missing time points:  [1]  which time point is missing:  [1]
27 24 [20, 35, 47, 54, 56, 57, 59, 61, 62, 66, 67, 74, 77, 79, 82, 83, 84, 89, 91, 93, 94, 95, 97, 98]
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum number of train epochs is reached
The maximum n

In [ ]:
len(res)

In [ ]:
n = 48
matrix = np.zeros((n,99))*-1
complete_all = ([int(x) - 1 for x in open('../indices_complete.txt','r').readline().split()])
comp_ind = list(map(int, list((np.array(complete_all)[::6]-3)/6)))
for i in range(n):
    matrix[i,res[i][0]] = res[i][1][res[i][0]]
ans_mean = np.apply_along_axis(lambda v: np.mean(v[v >= 0]), 0, matrix)
ans_std = np.apply_along_axis(lambda v: np.std(v[v >= 0]), 0, matrix)
ans_mean[np.isnan(ans_mean)] = 0
ans_std[np.isnan(ans_std)] = 0

data = np.vstack([gene_exp[:, i::6] for i in range(6)])
plt.figure(figsize=(12,6))
plt.plot(comp_ind,np.median(data, axis=0)[comp_ind],'o-', ms=10, linewidth=2)
plt.errorbar(comp_ind, ans_mean[comp_ind], yerr=ans_std[comp_ind], fmt='v-', ms=12, linewidth=2)

In [ ]:
# import pickle

# with open('./pickles/imputed_10.pkl', 'wb') as f:
#     pickle.dump(res, f)

In [ ]:
# complete_all = ([int(x) - 1 for x in open('../indices_complete.txt','r').readline().split()])
# comp_ind = list(map(int, list((np.array(complete_all)[::6]-3)/6)))

# import matplotlib
# matplotlib.rcParams['legend.frameon'] = False
# matplotlib.rcParams.update({'font.size':24})

# with open('./pickles/imputed_10.pkl', 'rb') as f:
#     res = pickle.load(f)
    
# mean_of_med = np.mean([res[i]['med'] for i in range(10)], axis=0)
# std_of_med = np.std([res[i]['med'] for i in range(10)], axis=0)
# data = np.median(np.vstack([gene_exp[:, i::6] for i in range(6)])[:, comp_ind], axis=0)

# plt.figure(figsize=(12,6))
# plt.plot(comp_ind, data, 'o-', ms=10, linewidth=2, label='observed')
# plt.errorbar(comp_ind, mean_of_med[comp_ind], yerr=std_of_med[comp_ind], fmt='v-', ms=12, linewidth=2, label='imputed')
# plt.xlabel('Gene')
# plt.ylabel('Median expression')
# plt.xlim(-1,99)
# plt.xticks([0,98],['0','98'])
# plt.ylim(0,0.4)
# plt.yticks([0,0.1,0.2,0.3,0.4],['0','','','','0.4'])
# plt.legend()
# # plt.savefig('./paper/imputation.pdf', boxx_inches='tight')
# plt.show()

In [ ]:
# import scipy

# plt.figure(figsize=(8,8))

# # reg = scipy.stats.linregress(saved, imputed)
# # x = np.linspace(0,1.5,3)
# # y = x*reg[0]
# saved = np.hstack([res[i]['saved'] for i in range(10)])
# imputed = np.hstack([res[i]['imputed'] for i in range(10)])
# imputed[imputed < 0] = 0
# plt.scatter(saved, imputed, alpha=0.05, s=3)
# plt.plot([0,1.6],[0,1.6], 'r--', linewidth=2, label='slope = 1')
# # plt.plot(x,y,'--', linewidth=2, label='regressed')
# plt.xlabel('observed')
# plt.ylabel('imputed')
# plt.xlim(0,1.6)
# plt.ylim(0,1.6)
# plt.xticks([0,0.5,1,1.5],['0','','','1.5'])
# plt.yticks([0,0.5,1,1.5],['0','','','1.5'])
# plt.legend()
# # plt.savefig('./paper/obvsimp.png', bbox_inches='tight')
# plt.show()

In [ ]:
# corr = np.mean([res[i]['corr'] for i in range(10)])

# print(corr)